In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Local/dcgan"

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Local/dcgan
Found GPU at: /device:GPU:0


Comment out the above if not using Google Colaboratory. 

## Define the Network
Below is the code used to create the DCGAN. Note that much of this code is taken from https://github.com/eriklindernoren/Keras-GAN. The main modifications here are some simplifications to variable names, added functionality for saving images, and the ability to plot the progression of the GAN over time on the same noise batch. 

In [0]:
from keras.datasets import mnist, cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.preprocessing import image as Image
import sys
import numpy as np
import os
import imageio

In [0]:
latent_dim = 32
channels =3
height =32
width = 32

##generator network
generator_input = Input(shape=(latent_dim, ))

x = Dense(128 * 16 * 16)(generator_input)
x = LeakyReLU()(x)
x = Reshape((16,16,128))(x)

x = Conv2D(256, 5, padding='same')(x)  # 16*16*256
x = BatchNormalization(momentum=0.8)(x)
x = LeakyReLU()(x)
x = UpSampling2D()(x)
#x = Conv2DTranspose(256, 4, strides=2, padding='same')(x) # 32*32*256 upsampling
x = LeakyReLU()(x)
x = Conv2D(256, 5, padding='same')(x)
x = BatchNormalization(momentum=0.8)(x)
x = LeakyReLU()(x)
x = Conv2D(256, 5, padding='same')(x)
x = BatchNormalization(momentum=0.8)(x)
x = LeakyReLU()(x)
x = Conv2D(channels, 7, activation='tanh', padding='same')(x) #sample 32*32*3 image

generator = Model(generator_input, x)
generator.summary()

#descriminator network
descriminator_input = Input(shape=(height, width, channels))
x = Conv2D(128, 3)(descriminator_input)
x = LeakyReLU()(x)
x = Dropout(0.25)(x)

x = Conv2D(128, 4, strides=2)(x)
x = BatchNormalization(momentum=0.8)(x)
x = LeakyReLU()(x)
x = Dropout(0.25)(x)

x = Conv2D(128, 4, strides=2)(x)
x = BatchNormalization(momentum=0.8)(x)
x = LeakyReLU()(x)
x = Dropout(0.25)(x)

x = Conv2D(128, 4, strides=2)(x)
x = LeakyReLU()(x)
x = Flatten()(x)
#x = Dense(128, activation='tanh')(x)
x = Dropout(0.4)(x)

x = Dense(1, activation='sigmoid')(x)
descriminator = Model(descriminator_input, x)
descriminator.summary()

#descriminator_optimizer = Adam(lr=0.0004,clipvalue=1.0, decay=1e-8)
descriminator_optimizer = Adam(lr=0.0002,clipvalue=1.0, decay=1e-8)

descriminator.compile(optimizer=descriminator_optimizer, loss='binary_crossentropy')

descriminator.trainable = False

#gan
gan_input = Input(shape=(latent_dim, ))
gan_output = descriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

#gan_optimizer = Adam(lr=0.0004,clipvalue=1.0, decay=1e-8)
gan_optimizer = Adam(lr=0.0002,clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')


#training the gan
(X_train, y_train), (_,_) = cifar10.load_data()
X_train  = X_train[y_train.flatten() == 6]
X_train = X_train.reshape((X_train.shape[0], )+(height, width, channels)).astype('float32')/255.

iterations = 4000
batch_size = 32
save_dir = "./gen1"
start = 0

for step in range(iterations):
    random_latent_vectors = np.random.normal(size = (batch_size, latent_dim)) #sample random points
    generated_images = generator.predict(random_latent_vectors) #output of the generator (decoded fake images)
    
    stop = start + batch_size
    real_images = X_train[start:stop]
    combined_images = np.concatenate([generated_images, real_images])#combine fake and real images
    
    labels = np.concatenate([np.zeros((batch_size, 1)), np.ones((batch_size, 1))])
    #labels += 0.005*np.random.random(size=labels.shape) #add noise to the labels
    
    d_loss = descriminator.train_on_batch(combined_images, labels) #train the descriminator
    
    random_latent_vectors = np.random.normal(size = (batch_size, latent_dim))#sample random vectors
    misleading_targets = np.ones((batch_size, 1)) #labels that lie that they are real images
    
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets) #train generator model via gan
    
    start += batch_size
    if start > len(X_train)-batch_size:
        start = 0
    
    if step %100 ==0:
        gan.save_weights('gan.h5')
        
        print("discriminator loss ",step," : ", d_loss)
        print("adverserial loss : ",step," : ", a_loss)
        
        img = Image.array_to_img(generated_images[0] * 255., scale=False)
        img.save("cifar10/generated_frog_"+str(step)+'.png')

        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_76 (InputLayer)        (None, 32)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_207 (LeakyReLU)  (None, 32768)             0         
_________________________________________________________________
reshape_28 (Reshape)         (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_202 (Conv2D)          (None, 16, 16, 256)       819456    
_________________________________________________________________
batch_normalization_156 (Bat (None, 16, 16, 256)       1024      
_________________________________________________________________
leaky_re_lu_208 (LeakyReLU)  (None, 16, 16, 256)       0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss  0  :  0.79147315
adverserial loss :  0  :  0.75722456
discriminator loss  100  :  7.196488e-05
adverserial loss :  100  :  0.012157657
discriminator loss  200  :  0.00011395703
adverserial loss :  200  :  0.00586361
discriminator loss  300  :  4.3258373e-05
adverserial loss :  300  :  0.0023058406
discriminator loss  400  :  2.9299707e-05
adverserial loss :  400  :  0.00422142
discriminator loss  500  :  1.7357868e-05
adverserial loss :  500  :  0.0017512488
discriminator loss  600  :  2.8087728e-05
adverserial loss :  600  :  0.001318652
discriminator loss  700  :  1.6755259e-05
adverserial loss :  700  :  0.00097054924
discriminator loss  800  :  7.988632e-06
adverserial loss :  800  :  0.0015461742


In [0]:
for step in range(iterations):
    random_latent_vectors = np.random.normal(size = (batch_size, latent_dim)) #sample random points
    generated_images = generator.predict(random_latent_vectors) #output of the generator (decoded fake images)
    
    stop = start + batch_size
    real_images = X_train[start:stop]
    combined_images = np.concatenate([generated_images, real_images])#combine fake and real images
    
    labels = np.concatenate([np.zeros((batch_size, 1)), np.ones((batch_size, 1))])
    #labels += 0.005*np.random.random(size=labels.shape) #add noise to the labels
    
    d_loss = descriminator.train_on_batch(combined_images, labels) #train the descriminator
    
    random_latent_vectors = np.random.normal(size = (batch_size, latent_dim))#sample random vectors
    misleading_targets = np.ones((batch_size, 1)) #labels that lie that they are real images
    
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets) #train generator model via gan
    
    start += batch_size
    if start > len(X_train)-batch_size:
        start = 0
    
    if step %10 ==0:
        gan.save_weights('gan.h5')
        
        print("discriminator loss ",step," : ", d_loss)
        print("adverserial loss : ",step," : ", a_loss)
        
        img = Image.array_to_img(generated_images[0] * 255., scale=False)
        img.save("cifar10/generated_frog_"+str(step)+'.png')

In [0]:
class DCGAN(): 
  def __init__(self, name= "mnist", imgrows=32, imgcols=32, channels= 3, latent_dim= 100):
    self.name= name
    self.img_rows= imgrows
    self.img_cols= imgcols
    self.channels= channels
    self.img_shape= (self.img_rows, self.img_cols, self.channels)
    self.latent_dim= latent_dim
    
    #parameters chosen from the paper by Radford et al.
    optimizer= Adam(0.0002, 0.5)
    
    #construct generator
    self.generator= self.generator()
    
    #construct discriminator
    self.discriminator= self.discriminator()
    self.discriminator.compile(loss= "binary_crossentropy",
                              optimizer= optimizer, 
                              metrics= ['accuracy'])
    
    #generator generates images from noise
    noise= Input(shape= (self.latent_dim, ))
    img= self.generator(noise)
    
    #only train generator
    self.discriminator.trainable= False
    
    #validity of image as determined by discriminator
    val= self.discriminator(img)
    
    #stack generator and discriminator
    self.GAN = Model(noise, val)
    self.GAN.compile(loss= "binary_crossentropy", optimizer= optimizer)
    
  
  def generator(self): 
    model = Sequential()
    model.add(Dense(128 * 16 * 16, activation= "relu", input_dim= self.latent_dim))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((16, 16, 128)))
    model.add(Conv2D(256, kernel_size= 5, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size= 3, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(256, 5, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(self.channels, kernel_size= 7, activation= 'tanh', padding= "same"))
    model.add(Activation("tanh"))
    print("Generator Summary ... ")
    model.summary()
    
    #input noise into generator
    noise= Input(shape= (self.latent_dim, ))
    #pass noise through model to retrieve image
    img= model(noise)
    
    return Model(noise, img)
  
  def discriminator(self): 
    model= Sequential()
    model.add(Conv2D(128, kernel_size= 3, strides= 2, input_shape= self.img_shape, padding= "same"))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size= 4, strides= 2, padding= "same"))
    model.add(ZeroPadding2D(padding= ((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size= 4, strides= 2, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=4, strides= 2, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation= "sigmoid"))
    print("Discriminator Summary ... ")
    model.summary()
    
    img= Input(shape= self.img_shape)
    val= model(img)
    return Model(img, val)
  
  def images(self, noise, rows= 5, cols= 5, name= "untitled", title= "",
             show= False): 
    generated= self.generator.predict(noise)
    
    #rescale images to [0, 1]
    generated= 0.5* generated + 0.5
    
    fig, ax= plt.subplots(rows, cols)
    count= 0
    for row in range(rows): 
      for col in range(cols): 
        img = Image.array_to_img(generated[count])
        ax[row, col].imshow(img)
        ax[row, col].axis("off")
        count += 1
    
    fig.suptitle(title)    
    fig.savefig("%s.png" % name)
    
    # Used to return the plot as an image rray
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    if not show: 
      plt.close()
    
    return image
    
      
  def train(self, images, epochs= 8000, batch_size= 32, 
           interval= 100, rows= 5, cols= 5):
    #purely for saving purposes ...
    fixed_noise_batch= np.random.normal(0, 1, (rows * cols, self.latent_dim))
    plots= []
    
    #if the directory specified by name doesn't exist, create it
    if not os.path.exists(self.name):
       os.makedirs(self.name)

    #adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    start= 0

    for epoch in range(epochs + 1):
      noise= np.random.normal(size= (batch_size, self.latent_dim))
      generated= self.generator.predict(noise)
      
      stop = start+ batch_size
      real= images[start: stop]
      combined_images= np.concatenate([generated, real])
      labels= np.concatenate([fake, valid])
      
      
      d_loss = self.discriminator.train_on_batch(combined_images, labels)
      g_loss = self.GAN.train_on_batch(noise, valid)
      
      start += batch_size
      if start > len(X_train)-batch_size:
        start = 0
        
      if epoch % interval == 0:
        status= "[%d] D loss: %f, acc: %.2f%% || G loss: %f" %(epoch, 
                 d_loss[0], d_loss[1] * 100, g_loss)
        print(status)
        
        #generate images
        name= "%s/%d" % (self.name, epoch)
        plot= self.images(name= name, noise= fixed_noise_batch, 
                          title= status, rows= rows, cols= cols)
        plots.append(plot)


## Run the Code
Below is the code to run the dcgan on the mnist dataset. 

In [64]:
(X_train, _), (_, _) = cifar10.load_data()
dcgan= DCGAN(name= "cifar10", imgrows= 32, imgcols= 32, channels= 3)

Generator Summary ... 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 32768)             3309568   
_________________________________________________________________
leaky_re_lu_199 (LeakyReLU)  (None, 32768)             0         
_________________________________________________________________
reshape_27 (Reshape)         (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_194 (Conv2D)          (None, 16, 16, 256)       819456    
_________________________________________________________________
batch_normalization_150 (Bat (None, 16, 16, 256)       1024      
_________________________________________________________________
leaky_re_lu_200 (LeakyReLU)  (None, 16, 16, 256)       0         
_________________________________________________________________
up_sampling2d_29 (UpSampling (None, 32, 32, 256)     

In [65]:
dcgan.train(images= X_train)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] D loss: 0.533711, acc: 62.50% || G loss: 0.421324
[100] D loss: 0.000161, acc: 100.00% || G loss: 0.000290
[200] D loss: 0.000089, acc: 100.00% || G loss: 0.000044
[300] D loss: 0.000204, acc: 100.00% || G loss: 0.000026
[400] D loss: 0.000024, acc: 100.00% || G loss: 0.000024
[500] D loss: 0.000010, acc: 100.00% || G loss: 0.000017


KeyboardInterrupt: ignored